In [ ]:
# text_generation_lstm.py
import tensorflow as tf
import numpy as np
import random
import sys
import os

# Download a small text corpus
path = tf.keras.utils.get_file("shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")
text = open(path, encoding="utf-8").read().lower()
print(f"Corpus length: {len(text)}")

# Create character-level vocabulary
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Prepare training sequences
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Build the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(maxlen, len(chars))),
    tf.keras.layers.Dense(len(chars), activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Train (short for demo)
model.fit(x, y, batch_size=128, epochs=3)

# Text generation helper
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Generate text
seed = "the moon shines bright tonight "
generated = seed
for i in range(400):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(seed[-maxlen:]):
        if char in char_indices:
            sampled[0, t, char_indices[char]] = 1.
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, 0.5)
    next_char = indices_char[next_index]
    generated += next_char
    seed += next_char

print("\nGenerated text:\n")
print(generated)


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Corpus length: 1115394
Number of sequences: 371778


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/3
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 261s 90ms/step - loss: 2.5734
Epoch 2/3
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 261s 90ms/step - loss: 1.9956
Epoch 3/3
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 264s 91ms/step - loss: 1.8272

Generated text:

the moon shines bright tonight 
;ssaet,oesosissesei.aeriyspited it my lakes to my larders
thee for my lord, and the mare the alle.

of ellarus:
that is the kith in the coultent thee and here,
and she there of our my dod he will where as with light,
and that i made thou have my spord, i me words:
why will brother hear dis of the dider of the have
the secoon of bather it a must the parters the with.

cloucester:
whise she the man
